In [1]:
# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "socrates"
# Replace with your application name (does not need to exist yet)
application = "hybridsearch"

In [2]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    HNSW,
    RankProfile,
    Component,
    Parameter,
    FieldSet,
    GlobalPhaseRanking,
    Function,
)

package = ApplicationPackage(
    name=application,
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(
                        name="title",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                    ),
                    Field(
                        name="body",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                        bolding=True,
                    ),
                    Field(
                        name="embedding",
                        type="tensor<float>(x[384])",
                        indexing=[
                            'input title . " " . input body',
                            "embed",
                            "index",
                            "attribute",
                        ],
                        ann=HNSW(distance_metric="angular"),
                        is_document_field=False,
                    ),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["title", "body"])],
            rank_profiles=[
                RankProfile(
                    name="bm25",
                    inputs=[("query(q)", "tensor<float>(x[384])")],
                    functions=[
                        Function(name="bm25sum", expression="bm25(title) + bm25(body)")
                    ],
                    first_phase="bm25sum",
                ),
                RankProfile(
                    name="semantic",
                    inputs=[("query(q)", "tensor<float>(x[384])")],
                    first_phase="closeness(field, embedding)",
                ),
                RankProfile(
                    name="fusion",
                    inherits="bm25",
                    inputs=[("query(q)", "tensor<float>(x[384])")],
                    first_phase="closeness(field, embedding)",
                    global_phase=GlobalPhaseRanking(
                        expression="reciprocal_rank_fusion(bm25sum, closeness(field, embedding))",
                        rerank_count=1000,
                    ),
                ),
            ],
        )
    ],
    components=[
        Component(
            id="e5",
            type="hugging-face-embedder",
            parameters=[
                Parameter(
                    "transformer-model",
                    {
                        "url": "https://github.com/vespa-engine/sample-apps/raw/master/examples/model-exporting/model/e5-small-v2-int8.onnx"
                    },
                ),
                Parameter(
                    "tokenizer-model",
                    {
                        "url": "https://raw.githubusercontent.com/vespa-engine/sample-apps/master/examples/model-exporting/model/tokenizer.json"
                    },
                ),
            ],
        )
    ],
)

In [3]:
from vespa.deployment import VespaCloud
import os

# Key is only used for CI/CD. Can be removed if logging in interactively
key = os.getenv("VESPA_TEAM_API_KEY", None)
if key is not None:
    key = key.replace(r"\n", "\n")  # To parse key correctly

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=application,
    key_content=key,  # Key is only used for CI/CD. Can be removed if logging in interactively
    application_package=package,
)

Setting application...
Running: vespa config set application ethantam33.hybridsearch
Setting target cloud...
Running: vespa config set target cloud

No api-key found for control plane access. Using access token.
Checking for access token in auth.json...
Access token expired. Please re-authenticate.
Your Device Confirmation code is: DSHF-CWRV
Automatically open confirmation page in your default browser? [Y/n] y
Opened link in your browser: https://login.console.vespa-cloud.com/activate?user_code=DSHF-CWRV
Waiting for login to complete in browser ... done;1m⣷
Success: Logged in
 auth.json created at /Users/ethantam/.vespa/auth.json
Successfully obtained access token for control plane access.
Certificate and key not found in /Users/ethantam/Desktop/personalCS/heartAI/src/.vespa or /Users/ethantam/.vespa/ethantam33.hybridsearch.default: Creating new cert/key pair with vespa CLI.
Generating certificate and key...
Running: vespa auth cert -N
Success: Certificate written to '/Users/ethantam/.

In [4]:
app = vespa_cloud.deploy()

Deployment started in run 1 of dev-aws-us-east-1c for ethantam33.hybridsearch. This may take a few minutes the first time.
INFO    [09:08:23]  Deploying platform version 8.478.26 and application dev build 1 for dev-aws-us-east-1c of default ...
INFO    [09:08:23]  Using CA signed certificate version 1
INFO    [09:08:24]  Using 1 nodes in container cluster 'hybridsearch_container'
INFO    [09:08:27]  Session 5 for tenant 'ethantam33' prepared and activated.
INFO    [09:08:47]  ######## Details for all nodes ########
INFO    [09:08:50]  h113421b.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [09:08:50]  --- platform vespa/cloud-tenant-rhel8:8.478.26
INFO    [09:08:50]  --- logserver-container on port 4080 has not started 
INFO    [09:08:50]  --- metricsproxy-container on port 19092 has not started 
INFO    [09:08:50]  h113421c.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [09:08:50]  --- platform vespa/cloud-tenant-rhel8:8.478.26
INFO    [09:08:50]  --- con

In [5]:
endpoint = vespa_cloud.get_mtls_endpoint()
endpoint

Found mtls endpoint for hybridsearch_container
URL: https://fafcfc3d.a2e5dc1c.z.vespa-app.cloud/


'https://fafcfc3d.a2e5dc1c.z.vespa-app.cloud/'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("BeIR/nfcorpus", "corpus", split="corpus", streaming=True)
vespa_feed = dataset.map(
    lambda x: {
        "id": x["_id"],
        "fields": {"title": x["title"], "body": x["text"], "id": x["_id"]},
    }
)